In [104]:
# pythonic version of the bash script instructions:
# for link to bash instructions, go to:
# https://gate.nmr.mgh.harvard.edu/wiki/bradmci/index.php/Individual_Reconstruction_%26_Analysis

# TO DO:
# need to add layer so I can run multiple subjects at the same time
# figure out subname problem .. symlink?
# create symlink within anat folder
# option to only ask for task name in first round
# automate par copying somehow by specifiying task order?
# enable: "issue a new project (each project has tasks connected to them__ BUT THESE TASKS COULD HAVE DIFFERENT PARAMS ETC>)"

In [105]:
def parse_findsession(findsession_output):
    session_values = {}
    for elements in findsession_output:
        try:
            [a, b] = elements.split(": ")
            a = a.strip()
            b = b.strip()
            if a in session_values:
                session_values[a].append(b)
            else:
                session_values[a]=[b]
        except ValueError:
            continue
    return session_values

In [106]:
def copyanything(src, dst):
    try:
        shutil.copytree(src, dst)
    except OSError as exc:
        if exc.errno == errno.ENOTDIR:
            shutil.copy(src, dst)
        else: raise

In [107]:
def maketxt(name, location, content):
    f = open(location+name, "w+")
    f.write(content)
    f.close()

In [108]:
def verifyrun(question):
    sys.stdout.write('%s [y/n]: ' %question)
    while True:
        try:
            return strtobool(input().lower())
        except ValueError:
            sys.stdout.write('Please respond with \'y\' or \'n\'.\n')
    

In [140]:
def parsenames(taskname, subjectfile):
    # read text file
    linkname = ''
    symlink = ''
    print(taskname)
    with open(subjectfile, 'r') as in_file:
        for line in in_file:
            editline = line.split()
            if editline[0] == taskname[4:]:
                linkname = editline[0][0:4]
                try:
                    symlink = editline[1]
                except IndexError:
                    symlink = ''
    in_file.close()
    return linkname, symlink

In [152]:
import getpass

def passprompt():
    USER= input("Please enter your USERNAME for launchpad acess: ")
    print('PASSWORD: ')
    PASS= getpass.getpass()
    return USER, PASS
    

In [125]:
# load in all modules and pathways
from distutils.util import strtobool
import sys
import os
from os import system
from os.path import abspath

from nipype import Workflow, Node, MapNode, Function
from nipype.interfaces.fsl import BET, IsotropicSmooth, ApplyMask
from nipype.interfaces.freesurfer import MRIConvert

from nilearn.plotting import plot_anat
#%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import csv
import re
import shutil, errno

home = '/autofs/homes/002/rje11/'
mydirectory = '/autofs/cluster/animal/users/rezzo/'

####### USER INPUT NEEDED TO RUN SCRIPT: #######

# TASK: name this after task substring in scaninfo file (e.g. 'FPW' or'MT' or AAScout). 
task = ['FPW', 'MT'] 

# file with list [COL1 = subject IDs (e.g. ITAY_F_34_201701220); COl2 = recon directory]
subjectfile = '/autofs/homes/002/rje11/FuncLoc_Sublist'

# make subjectlist, e.g. ['PCA_ITAY_F_34', 'PCA_RBAL_F_49'] from subjectfile
subjectlist =  ['PCA_CWIL_F_44']   # temp

unpackedtype = 'nii'

In [ ]:
# find path to dicoms

for indiv in subjectlist:

    findsession_output = !findsession $indiv
    
    # create dictionary with listed values (corresponding to each session)
    #session_values = parse_findsession(findsession_output)
    session_values = parse_findsession(findsession_output)
    
    find = re.compile(r"^[^_]*")
    projectname = re.search(find, indiv).group(0)
    
    try:
        # create a directory for unique session
        projectdir = mydirectory+projectname
        os.makedirs(projectdir)
    except FileExistsError:
        print(projectdir+' directory exists')

In [127]:
session_values

{'SUBJECT': ['PCA_CWIL_F_44_20181113'],
 'SUBJ ID': ['18.11.13-13:30:04-STD-1.3.12.2.1107.5.2.32.35162'],
 'DATE': ['November 13, 2018'],
 'TIME': ['14:01:51'],
 'OPERATR': ['rf699:tan (bay3) [D]'],
 'XPRMNTR': [''],
 'PATH': ['/cluster/archive/323/siemens/TrioTim-35162-20181113-135146-001470']}

In [128]:
# for each session (5-10 min PER RUN)
for elements in range(0, len(session_values["SUBJECT"])): # define range as number of sessions
    srcdir = session_values["PATH"][elements]+"/"
    SESSION_DIR = projectdir+"/"+session_values["SUBJECT"][elements]+'/'
    dicomdir = SESSION_DIR+"dicoms/"
    taskname = session_values["SUBJECT"][elements]
    anatdir = SESSION_DIR+"anat/"
    
    # parse take name to obtain the recon directory
    # use the four letter name as symlink name
    [linkname, symlink] = parsenames(taskname, subjectfile)
    
    try:
        # make directories for each session if not already existing
        os.makedirs(anatdir)
        os.makedirs(dicomdir)

        #load in the anatomical data
        if symlink == '':
            warn = 'RECON directory is missing. Please add as symlink to subjects ANAT folder later.'
            print(warn)
            system('echo ---------- RECON directory missing. Please add as symlink to ANAT folder -------')
            maketxt('readme',anatdir,warn)
            #add a readme.txt instructing how to add recon symlink?
        else:
            cmdstring = 'ln -s %s %s' % (linkname, anatdir+symlink) 
            system(cmdstring)
            print(cmdstring)

        # put subjectname textfile into main folder !!!!!!!!!!!!!!!!!!
        if linkname == "":
            warn = 'SUBJECT cannot be found on the subjectfile. Please add the subject initials to the subjectname textfile before analysis.'
            print(warn)
            system('echo ---------- SUBJECT cannot be found on the subjectfile. Please add the subject initials to the subjectname textfile.')
        maketxt('subjectname', SESSION_DIR,linkname)

    except FileExistsError:
        print("directory already exists, proceeding with unpacking")
        system('echo ------directory already exists, proceeding with unpacking------')

    #make temp directory and unpack first dicom to chck if exists
    cmdstring = 'mkdir %stmpdir'% (SESSION_DIR)
    system(cmdstring)
    firstfile = os.listdir(srcdir)[0]
    cmdstring = "rsync %s %s" % (srcdir+firstfile, SESSION_DIR+'tmpdir/')
    system(cmdstring)
    namecheck = os.listdir(SESSION_DIR+'tmpdir/')[0]

    #check if file exists in dicom directory (if one is there that means all 
    # scans have been unpacked: bold, MEMPRAGE, etc.)
    print(namecheck)
    if os.path.isfile(dicomdir+namecheck):
        print('dicoms appear to be unpacked already')
        system('echo --------dicoms appear to be unpacked already.. proceeding--------')
        # delete tmpdir
        cmdstring = "rm -rf %s" % (SESSION_DIR+'tmpdir/')
        system(cmdstring)
    else:
        cmdstring = "rsync %s %s" % (srcdir+'*', dicomdir)
        system(cmdstring)
        print(cmdstring)
        system('echo -------copying dicoms to dicom directory from source-------')
        # delete tmpdir
        cmdstring = "rm -rf %s" % (SESSION_DIR+'tmpdir/')
        system(cmdstring)

    # check if scan.info exists (same for all files within session)
    exists = os.path.isfile(SESSION_DIR+'/scan.info') # checks for scan.info file
    if exists==0:
        # unpack these dicoms
        cmdstring = 'unpacksdcmdir -src %s -targ %s -scanonly %s/scan.info' % (srcdir, SESSION_DIR, SESSION_DIR)
        system(cmdstring)
        print(cmdstring)
        system('echo ------unpacking scan info from dicoms-----')
        #make lines low into one function
        
    exists = os.path.isfile(SESSION_DIR+'/scaninfo.csv') # checks for parsed scaninfo.csv file
    if exists==0:
        with open(SESSION_DIR+'/scan.info', 'r') as in_file:
            print("Parsing scan.info to scaninfo.csv ...")
            system('echo -------Parsing scan.info to scaninfo.csv ...-------')
            for line in in_file:
                # make list out of elements in each line
                editline = line.split()
                with open(SESSION_DIR+"/scaninfo.csv", 'a') as result:
                    wr = csv.writer(result, dialect='excel')
                    wr.writerow(editline)
                result.close()
        in_file.close()
        print("COMPLETED PARSING SCANINFO FILE")
        system('echo ------COMPLETED PARSING SCANINFO FILE------')
            
    # import scan.info from SESSION_DIR as pandas dataframe
    scan_info = pd.read_csv(SESSION_DIR+'/scaninfo.csv', header=None)
    
    # for each of the confirmed runs, make a bold_directory for task
    for element in task:
        # initialize indexing list to later add to cmdstrng
        run_list = []
        scan_name_index = []
        scan_num_index = []
        runsdir = "/bold_"+element
        try:
            os.makedirs(SESSION_DIR+runsdir)
            for row in range(0, len(scan_info.index)):
                # index values of every row with inclusion criteria (obain task name and run #)
                if element in scan_info.iloc[row,1]:
                    print('Currently unpacking functional runs for '+element)
                    if userinput == 1:
                        scan_name_index.append(scan_info.iloc[row,1])
                        scan_num_index.append(scan_info.iloc[row,0])
                    else:
                        #exit the whole loop
                        # ask for confirmation of task contents (these are all the runs)
                        instring = 'Would you like to include run '+str(scan_info.iloc[row,1])+' for task '+str(element)+' ?'
                        userinput = verifyrun(instring)

            for name in range(0, len(scan_name_index)):
                print(scan_name_index[name])
                temp = ' -run '+ str(scan_num_index[name]) + ' '+ runsdir+ ' ' + unpackedtype + ' ' + 'f.'+unpackedtype
                run_list.append(temp) 

            #combine list of strings into one long string about run info
            runstring = "".join(run_list)
            cmdstring = 'unpacksdcmdir -src %s -targ %s %s' % (srcdir, SESSION_DIR ,runstring) # directory sturture may be messed up!
            print(cmdstring)
            system('echo ------------unpacking raw functional data--------------')
            system(cmdstring)
        except FileExistsError:
            print("functional directory already exists and is unpacked")
            system('echo -------functional directory already exists and is unpacked------')
        
print("------------------------- UNPACKING COMPLETE --------------------------------")
system('echo ------------------------- UNPACKING COMPLETE --------------------------------')


PCA_CWIL_F_44_20181113
RECON directory is missing. Please add as symlink to subjects ANAT folder later.
MR.1.3.12.2.1107.5.2.32.35162.2018111315015396326464805
rsync /cluster/archive/323/siemens/TrioTim-35162-20181113-135146-001470/* /autofs/cluster/animal/users/rezzo/PCA/PCA_CWIL_F_44_20181113/dicoms/
unpacksdcmdir -src /cluster/archive/323/siemens/TrioTim-35162-20181113-135146-001470/ -targ /autofs/cluster/animal/users/rezzo/PCA/PCA_CWIL_F_44_20181113/ -scanonly /autofs/cluster/animal/users/rezzo/PCA/PCA_CWIL_F_44_20181113//scan.info
Parsing scan.info to scaninfo.csv ...
COMPLETED PARSING SCANINFO FILE
Currently unpacking functional runs for FPW
Currently unpacking functional runs for FPW
fMAP_FPW_3mm_TR2_ISO_TP169
fMAP_FPW_3mm_TR2_ISO_TP169
unpacksdcmdir -src /cluster/archive/323/siemens/TrioTim-35162-20181113-135146-001470/ -targ /autofs/cluster/animal/users/rezzo/PCA/PCA_CWIL_F_44_20181113/  -run 9 /bold_FPW nii f.nii -run 11 /bold_FPW nii f.nii
Currently unpacking functional runs

0

In [239]:
def identifyfolders(stringname, directory, N):
    dirlist = list(os.listdir(directory))
    if N == 1:
        find = re.compile(re.escape(stringname))
    if N == 0:
        find = re.compile(r"^(?<!\d)\d{3}(?!\d)")
    finallist = []

    # list of all folders in directory with particular string
    for file in dirlist:
        if re.search(find, file):
            finallist.append(file)
    return finallist

In [264]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]

In [277]:
# Add paradigm files to every run folder throughout session folders
# NOTE: INITIAL PARADIGM FILES MUST BE MANUALLY CREATED AND ADDED TO PARFILES FOLDER
# WARNING: this section overwrites existing paradigm files; assuming this is ground truth.

par_MT = ['/autofs/cluster/animal/users/rezzo/parfiles/MT/run1/paradigm.par']
par_FPW = ['/autofs/cluster/animal/users/rezzo/parfiles/FPW/run1/paradigm.par','/autofs/cluster/animal/users/rezzo/parfiles/FPW/run2/paradigm.par']

# look in the Project folder to list all sessions
sessionslist = identifyfolders("PCA", projectdir, 1)

# go into each session and look for bold folders
for session in sessionslist:
    bolddir = identifyfolders("bold", projectdir+'/'+session+'/', 1)
    print(session)
    
    # go into each bold folder and look for run folders
    for bold in bolddir:
        print(bold)
        runs = identifyfolders("0", projectdir+'/'+session+'/'+bold+'/', 0)
        
        #put runs in order of aquisition
        runs.sort(key=natural_keys)
        print(runs)
        
        if bold == 'bold_MT':
            parlist = par_MT
        elif bold == 'bold_FPW':
            parlist = par_FPW
        else:
            parlist = []
        counter = 0
                  
        # create paradigm.par file for every run
        for run in runs:
            if counter == len(parlist):
                counter = 0
            try:
                cmdstring = 'cp %s %s' % (parlist[counter], projectdir+'/'+session+'/'+bold+'/'+run+'/')
                print(cmdstring)
                system(cmdstring)
                counter = counter+1
            except(IndexError):
                print('ERROR: skipping ' +session+'/'+bold+'/'+run+ '. Check if bold folder is incorrectly named. If this is a new bold task, you must create a new list that includes directories for each run`s paradigmfile. Note that the directories should be in order by run. See e.g. par_FPW.')
        if runs == []:
            print(bold+" folder is empty.")

PCA_CWIL_F_44_20181113
bold_FPW
['009', '011']
cp /autofs/cluster/animal/users/rezzo/parfiles/FPW/run1/paradigm.par /autofs/cluster/animal/users/rezzo/PCA/PCA_CWIL_F_44_20181113/bold_FPW/009/
cp /autofs/cluster/animal/users/rezzo/parfiles/FPW/run2/paradigm.par /autofs/cluster/animal/users/rezzo/PCA/PCA_CWIL_F_44_20181113/bold_FPW/011/
bold_MT
['010']
cp /autofs/cluster/animal/users/rezzo/parfiles/MT/run1/paradigm.par /autofs/cluster/animal/users/rezzo/PCA/PCA_CWIL_F_44_20181113/bold_MT/010/
PCA_WSTA_M_56_20170802
bold_FPW
['009', '011']
cp /autofs/cluster/animal/users/rezzo/parfiles/FPW/run1/paradigm.par /autofs/cluster/animal/users/rezzo/PCA/PCA_WSTA_M_56_20170802/bold_FPW/009/
cp /autofs/cluster/animal/users/rezzo/parfiles/FPW/run2/paradigm.par /autofs/cluster/animal/users/rezzo/PCA/PCA_WSTA_M_56_20170802/bold_FPW/011/
bold_MT
['010']
cp /autofs/cluster/animal/users/rezzo/parfiles/MT/run1/paradigm.par /autofs/cluster/animal/users/rezzo/PCA/PCA_WSTA_M_56_20170802/bold_MT/010/
PCA_PTHO

PCA_PTHO_M_56_20180807
bold_FPW
['011', '012']
cp /autofs/cluster/animal/users/rezzo/parfiles/FPW/run1/paradigm.par /autofs/cluster/animal/users/rezzo/PCA/PCA_PTHO_M_56_20180807/bold_FPW/011/
cp /autofs/cluster/animal/users/rezzo/parfiles/FPW/run2/paradigm.par /autofs/cluster/animal/users/rezzo/PCA/PCA_PTHO_M_56_20180807/bold_FPW/012/
bold_MT
[]
bold_MT folder is empty.


In [153]:
# SSH onto launchpad to create batch scripts

import pysftp

[USER, PASS] =passprompt()

with pysftp.Connection('launchpad', username=USER, password=PASS) as sftp:
    print(sftp.isdir('/autofs/cluster/animal/scan_data/leads/'))

Please enter your USERNAME for launchpad acess: rje11
PASSWORD: 
········
True


In [ ]:
# need to change bold folder names as I do analysis.

In [83]:
# in terminal::
# FOR MT

# change sessidfile to have the subject of interest

# this one is linked to /space/weck/2/users/MR/ITAY/
setenv SUBJECTS_DIR /autofs/cluster/animal/users/rezzo/PCA/PCA_MDOY_F_65_20171219/anat/
setenv SUBJECT MDOY

# # WARNING" these create files with specific anatomical calls (make sure to set variable); native is in native functional space (not native structural)
mkanalysis-sess -fsd bold -analysis firstlevelMT.lh -surface $SUBJECT lh -fwhm 5 -paradigm paradigm.par -event-related -spmhrf 0 -refeventdur 25 -polyfit 2 -nskip 4 -TR 2 -nconditions 2 -force
mkanalysis-sess -fsd bold -analysis firstlevelMT.rh -surface $SUBJECT rh -fwhm 5 -paradigm paradigm.par -event-related -spmhrf 0 -refeventdur 25 -polyfit 2 -nskip 4 -TR 2 -nconditions 2 -force
mkanalysis-sess -fsd bold -native -analysis firstlevelMT.whole -fwhm 5 -paradigm paradigm.par -event-related -spmhrf 0 -refeventdur 25 -polyfit 2 -nskip 4 -TR 2 -nconditions 2 -force

mkcontrast-sess -analysis firstlevelMT.rh -contrast moving-vs-still -a 1 -c 2
mkcontrast-sess -analysis firstlevelMT.lh -contrast moving-vs-still -a 1 -c 2
mkcontrast-sess -analysis firstlevelMT.whole -contrast moving-vs-still -a 1 -c 2

# !!!!!!!!!!!!!! REPLACE SUBJECT NAME
preproc-sess -s PCA_MDOY_F_65_20171219 -fsd bold -surface $SUBJECT lhrh -per-run -fwhm 5 -force

# !!!!!!!!!!!!!!!! REPLACE SESSION ID FILE
## needs to run on 'bold'? Also the visualizations do
## make a file with a list of all sessions in project folder
selxavg3-sess -s PCA_MDOY_F_65_20171219 -analysis firstlevelMT.lh -overwrite
selxavg3-sess -s PCA_MDOY_F_65_20171219 -analysis firstlevelMT.rh -overwrite
selxavg3-sess -s PCA_MDOY_F_65_20171219 -analysis firstlevelMT.whole -overwrite

# visualize surface by hemisphere
tksurfer-sess -s PCA_MDOY_F_65_20171219 -analysis firstlevelMT.lh -c moving-vs-still # for surface
tksurfer-sess -s PCA_MDOY_F_65_20171219 -analysis firstlevelMT.rh -c moving-vs-still

# FOR VOLUME JAN
#setenv SUBJECTS_DIR /autofs/cluster/animal/scan_data/dl-xnat/MASTER/ITAY_F_34/PCA_ITAY_F_34_20171220/recons/FS6_004_006/2017-12-21T17-10-11-115/
tkmedit-sess -s PCA_MDOY_F_65_20171219 -analysis firstlevelMT.whole -c moving-vs-still


'PCA_ITAY_F_34_20171220'

In [ ]:
# FOR FPW

# change sessidfile to have the subject of interest


# this one has a recursive symlink
setenv SUBJECTS_DIR /autofs/cluster/animal/users/rezzo/recons/ITAY_F_34_20171220/
setenv SUBJECT edit.FS6_004_006

mkanalysis-sess -fsd bold -native -analysis firstlevelFPW.lh -surface $SUBJECT lh -fwhm 5 -paradigm paradigm.par -event-related -spmhrf 0 -refeventdur 8 -polyfit 2 -nskip 4 -TR 2 -nconditions 4 -force
mkanalysis-sess -fsd bold -native -analysis firstlevelFPW.rh -surface $SUBJECT rh -fwhm 5 -paradigm paradigm.par -event-related -spmhrf 0 -refeventdur 8 -polyfit 2 -nskip 4 -TR 2 -nconditions 4 -force
mkanalysis-sess -fsd bold -native -analysis firstlevelFPW.whole -fwhm 5 -paradigm paradigm.par -event-related -spmhrf 0 -refeventdur 8 -polyfit 2 -nskip 4 -TR 2 -nconditions 4 -force

mkcontrast-sess -analysis firstlevelFPW.rh -contrast faces-vs-placeswords -a 1 -c 4 -c 2
mkcontrast-sess -analysis firstlevelFPW.lh -contrast faces-vs-placeswords -a 1 -c 4 -c 2
mkcontrast-sess -analysis firstlevelFPW.whole -contrast faces-vs-placeswords -a 1 -c 4 -c 2

mkcontrast-sess -analysis firstlevelFPW.rh -contrast places-vs-faceswords -a 4 -c 1 -c 2
mkcontrast-sess -analysis firstlevelFPW.lh -contrast places-vs-faceswords -a 4 -c 1 -c 2
mkcontrast-sess -analysis firstlevelFPW.whole -contrast places-vs-faceswords -a 4 -c 1 -c 2

preproc-sess -s PCA_ITAY_F_34_20171220 -fsd bold -surface $SUBJECT lhrh -per-run -fwhm 5 -force

# ADD REGISTATION?


# !!!!!!!!!!!!!!!!1 REPLACE SESSION ID FILE
## make a file with a list of all sessions in project folder
selxavg3-sess -sf sessidfile -analysis firstlevelFPW.lh -overwrite
selxavg3-sess -sf sessidfile -analysis firstlevelFPW.rh -overwrite
selxavg3-sess -sf sessidfile -analysis firstlevelFPW.whole -overwrite

# visualize surface by hemisphere
tksurfer-sess -s PCA_ITAY_F_34_20171220 -analysis firstlevelFPW.lh -c faces-vs-placeswords # for surface
tksurfer-sess -s PCA_ITAY_F_34_20171220 -analysis firstlevelFPW.rh -c faces-vs-placeswords

# FOR VOLUME
#setenv SUBJECTS_DIR /autofs/cluster/animal/scan_data/dl-xnat/MASTER/ITAY_F_34/PCA_ITAY_F_34_20171220/recons/FS6_004_006/2017-12-21T17-10-11-115/
tkmedit-sess -s PCA_ITAY_F_34_20171220 -analysis firstlevelFPW.whole -c faces-vs-placeswords



In [ ]:
# to extract stats from ROI for volume (1 condition)

# this one is linked to /space/weck/2/users/MR/ITAY/
setenv SUBJECTS_DIR /autofs/cluster/animal/users/rezzo/recons/ITAY_F_34_20170110/
setenv SUBJECT edit.FS6_004_019

# # this one has a recursive symlink
# setenv SUBJECTS_DIR /autofs/cluster/animal/users/rezzo/recons/ITAY_F_34_20171220/
# setenv SUBJECT edit.FS6_004_006

setenv CON /autofs/cluster/animal/users/rezzo/PROJECT/PCA_ITAY_F_34_20170110/bold/firstlevel.whole/moving-vs-still/ces.nii.gz
setenv SIG /autofs/cluster/animal/users/rezzo/PROJECT/PCA_ITAY_F_34_20170110/bold/firstlevel.whole/moving-vs-still/sig.nii.gz
setenv REG /autofs/cluster/animal/users/rezzo/PROJECT/PCA_ITAY_F_34_20170110/bold/register.dof6.lta

mri_vol2vol --mov $CON --reg $REG --fstarg --interp nearest --o ces.moving.anat.bb.mgh 
mri_vol2vol --mov $SIG --reg $REG --fstarg --interp nearest --o sig.moving.anat.bb.mgh 

mri_segstats --seg $SUBJECTS_DIR/$SUBJECT/mri/aparc+aseg.mgz --ctab $FREESURFER_HOME/FreeSurferColorLUT.txt --id 1007 --id 1015 --id 2015 --i ces.moving.anat.bb.mgh --sum ces.moving.bb.stats
mri_segstats --seg $SUBJECTS_DIR/$SUBJECT/mri/aparc+aseg.mgz --ctab $FREESURFER_HOME/FreeSurferColorLUT.txt --id 1007 --id 1015 --id 2015 --i ces.moving.anat.bb.mgh --sum ces.moving.abs-masked.bb.stats --mask sig.moving.anat.bb.mgh --maskthresh 2 --masksign abs

